<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course5/week1/Building_a_Recurrent_Neural_Network_Step_by_Step_v3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building your Recurrent Neural Network - Step by Step

5주차 첫 번째 과제에 오신것을 환영합니다! 이번 과제에서, 여러분들은 numpy를 사용하여 Recurrent Neural Network(순환 신경망: RNN) 의 핵심 구성 요소를 구현해볼 것입니다.

RNN(Recurrent Neural Networks)은 "메모리"가 있기 때문에 자연어 처리 및 기타 시퀀스 작업에 매우 효과적입니다. 한 번에 하나씩 입력 $x ^ {\langle t \rangle}$ (예 : 단어)를 읽고 한 time step에서 다음 time step으로 전달되는 은닉층 활성화 값을 통해 해당 time step의 데이터에 대한 정보/컨텍스트를 기억할 수 있습니다. 이를 통해 단방향 RNN이 과거의 정보를 가져와 추후에 들어오는 입력을 처리 할 수 있습니다. 양방향 RNN은 과거와 미래 모두에서 정보 혹은 컨텍스트를 가져올 수 있습니다.

**표기법**:
- 위첨자 $[l]$는 $l$번째 레이어와 관련된 개체를 나타냅니다.
- 위첨자 $(i)$는 훈련 데이터 세트의 $i$번째 데이터와 관련된 객체를 나타냅니다.
- 위첨자 $\langle t \rangle$은 $t$ 번째 time step에 있는 객체를 나타냅니다.
- 아래첨자 $i$는 벡터의 $i$ 번째 요소를 나타냅니다.

예를 들어,
- $a^{(2)[3]<4>}_5$ 라는 값은 는 두 번째 훈련 예제(2), 세 번째 레이어[3], 네 번째 time stop<4> 및 벡터의 다섯 번째 항목의 활성화 변수를 나타냅니다.

<br>

#### Pre-requisites
- 이 과제는 여러분들이 이미 `numpy` 라이브러리에 익숙하다고 가정하고 진행됩니다.
- numpy에 대한 지식을 리프레시하고 싶다면, 이 Specialization 코스의 1번 코스인 "Neural Networks and Deep Learning"를 검토 할 수 있습니다.
  - 특히, 2 주차 과제 [ "Python Basics with numpy (선택 사항)"](https://www.coursera.org/learn/neural-networks-deep-learning/item/Zh0CU)를 다시 수행하세요.

<br>

#### 시작 코드를 수정할 때주의하세요
- 채점 대상이 되는 함수를 작업 할 때는 다음 주석 사이에 있는 코드 만 수정해야합니다.
```python
#### START CODE HERE
```
과
```python
#### END CODE HERE
```
* 특히, 채점 대상이 되는 루틴의 첫 번째 줄을 수정하지 않도록 주의하십시오. 다음으로 시작합니다.
```python
# GRADED FUNCTION : routine_name
```
- 자동 채점 알고리즘(autograder)은 특정 함수를 찾기 위해 다음 사항이 필요합니다.
- 줄 간격/텍스트 간격이 변경되어오 자동 채점 알고리즘에 문제가 발생합니다.
- 위의 사항들을 수정했거나, 필수 사항이 일부 누락 된 경우 'failed'를 반환합니다.

이번 과제를 수행하는데 필요한 패키지들을 불러오는 것으로 이번 과제를 시작해봅시다.

In [ ]:
import numpy as np
from rnn_utils import *

## 1 - Forward propagation for the basic Recurrent Neural Network

이번 주차 마지막에, 여러분은 RNN을 사용한 음악을 만들어볼 예정입니다. 앞으로 여러분이 구현할 RNN의 기본적인 구조는 아래 그림과 같습니다. 아래의 예시는 $T_x = T_y$인 예시입니다.

<img src="arts/RNN.png" style="width:500;height:300px;">
<center>그림 1 : 기본적인 RNN 모델</center>


### Dimensions of input $x$

#### Input with $n_x$ number of units

- 단일 입력 데이터의 단일 time step의 경우, $x^{(i) \langle t \rangle}$는 1차원 input 벡터입니다.
- 언어를 하나의 예시로 삼아보자면, 5000개의 단어를 가진 언어는 5000개의 unit을 가진 one-hot encoding 벡터로 나타낼 수 있습니다. 따라서 $x^{(i) \langle t \rangle}$는 `(5000, )` 의 shape를 갖습니다.
- $n_x$ 표기를 사용해 단일 학습 데이터의 단일 time step에서의 unit의 개수를 표현합니다. 

#### Time steps of size $T_{x}$
- 순환 신경망 RNN에는 여러 time step이 있으며 특정 time step을 $t$로 나타냅니다.
- 영상 강의에서, $ x^{(i)}$ 가 여러 time step $T_x$로 구성된 단일 학습 데이터를 보았습니다. 예를 들어 10 개의 time step이 있는 경우 $ T_{x} = 10$ 입니다.

#### Batches of size $m$
- 각각 20 개의 훈련 데이타가있는 미니 배치가 있다고 가정 해 보겠습니다.
- 벡터화의 이점을 얻기 위해 $x^{(i)}$ 예제의 열 20 개를 쌓아 올립니다.
- 예를 들어, 위 경우의 텐서는 `(5000, 20, 10)`의 shape를 가집니다.
- 미니 배치에 포함된 훈련 데이터의 수를 나타 내기 위해 $m$를 사용합니다.
- 따라서 미니 배치의 shape는 `(n_x, m, T_x)`입니다.

#### 3D Tensor of shape $(n_{x},m,T_{x})$
- $(n_x, m, T_x)$ 모양의 3 차원 텐서 $x$는 RNN에 투입되는 $x$ 입력을 나타냅니다.

#### Taking a 2D slice for each time step: $x^{\langle t \rangle}$
- 각 time step에서 훈련 데이터의 미니 배치를 사용합니다 (단일 예제를 사용하지 않는다는 것에 주의하세요).
- 따라서 각 시간 단계 $t$에 대해 $(n_x, m)$ shape의 2D 슬라이스를 사용합니다.
- 우리는이 2D 슬라이스를 $x^{\langle t \rangle} $ 라고 합니다. 코드 변수 이름은 `xt` 입니다.

### Definition of hidden state $a$
- 한 time step에서 다른 time step까지 RNN으로 전달되는 활성화 $a^{\langle t \rangle}$ 값을 "hidden state"라고합니다.

### Dimensions of hidden state $a$

- 입력 텐서 $ x $와 유사하게 단일 학습 예제의 hidden state는 길이가 $ n_ {a} $ 인 벡터입니다.
- 미니 배치가 $m$개의 훈련 데이터를 가지는 경우, 미니 배치의 shape는 $ (n_ {a}, m) $입니다.
- time step 차원을 포함 할 때 hidden state의 shape는 $ (n_ {a}, m, T_x) $입니다.
- 인덱스 $ t $를 사용하여 time step를 반복하고 3D 텐서의 2D 슬라이스로 작업합니다.
- 이 2D 슬라이스를 $ a ^ {\langle t \rangle} $라고 합니다.
- 코드에서 사용하는 변수 이름은 구현되는 함수에 따라 `a_prev` 또는 `a_next`입니다.
- 이 2D 슬라이스의 shape는 $ (n_ {a}, m) $입니다.

### Dimensions of prediction $\hat{y}$

- 입력 및 hidden state와 유사하게 $ \ hat {y} $는 $ (n_ {y}, m, T_ {y}) $ 모양의 3D 텐서입니다.
     - $ n_ {y} $ : 예측을 나타내는 벡터의 단위 수.
     - $ m $ : 미니 배치의 예 수.
     - $ T_ {y} $ : 예측의 시간 단계 수.
- 단일 time step $ t $의 경우 2D 슬라이스 $ \hat {y} ^ {\langle t \rangle} $의 모양은 $ (n_ {y}, m) $입니다.
- 코드에서 사용하는 변수 이름은 다음과 같습니다.
     - `y_pred` : $ \hat {y} $
     - `yt_pred` : $ \hat {y} ^ {\langle t \rangle} $

RNN을 구현하는 방법은 다음과 같습니다.

**Step** :
1. RNN의 한 time step에 필요한 계산을 구현합니다.
2. 모든 입력을 한 번에 하나씩 처리하기 위해 $T_x$ time step에 대한 루프를 구현합니다.

### 1.1 - RNN cell

순환 신경망 RNN은 단일 cell을 반복적으로 사용한다는 것을 볼 수 있습니다. 먼저 단일 time step에 대한 계산을 구현합니다. 다음 그림은 RNN 셀의 단일 time step에 대한 작업을 설명합니다.

<img src="arts/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">
<center>그림 2 : 기본적인 RNN cell. $ x ^ {\langle t \rangle} $ (현재 입력) 및 $ a ^ {\langle t-1 \rangle} $ (과거 정보를 포함하는 이전 hidden state)를 입력으로 취하고 $ a ^ {\langle t \rangle} $을 출력합니다. 이는 다음 RNN cell에 입력으로 제공되며 $ \hat {y} ^ {\langle t \rangle} $ 예측에도 사용됩니다.</center>

<br>

#### RNN cell versus `rnn_cell_forward`
- RNN 셀은 hidden state $ a ^ {\langle t \rangle} $을 출력합니다.
  - RNN cell은 아래 그림에서 실선이있는 안쪽 상자로 나타낼 수 있습니다.
- 우리가 구현할 함수`rnn_cell_forward`는 예측 결과인 $ \hat {y} ^ {\langle t \rangle} $을 계산합니다.
  - `rnn_cell_forward` 는 그림에서 파선이 있는 바깥쪽의 상자로 나타낼 수 있습니다.

**연습 문제** : 그림 2에 설명된 RNN 셀을 구현합니다.

**지시 사항**:
1. tanh 활성화로 hidden unit을 계산합니다. $ a ^ {\langle t \rangle} = \tanh (W_ {aa} a ^ {\langle t-1 \rangle} + W_ {ax} x ^ {\langle t \rangle} + b_a) $.
2. 새로운 hidden unit $ a ^ {\langle t \rangle} $을 사용하여 $ \hat {y} ^ {\langle t \rangle} = softmax (W_ {ya} a ^ {\langle t \rangle} + b_y) $를 계산하세요. 사전에 제공되는 `softmax` 함수를 사용하세요.
3. $ (a ^ {\langle t \rangle}, a ^ {\langle t-1 \rangle}, x ^ {\langle t \rangle}, parameters) $를 `cache`에 저장합니다.
4. 최종적으로 $ a ^ {\langle t \rangle} $, $ \ hat {y} ^ {\langle t \rangle} $ 및 `cache`를 반환합니다.

#### 추가 힌트
* [numpy.tanh](https://www.google.com/search?q=numpy+tanh&rlz=1C5CHFA_koUS854US855&oq=numpy+tanh&aqs=chrome..69i57j0l5.1340j0j7&sourceid=chrome&ie=UTF-8)
* 제공되는 `softmax`함수를 사용하세요. `rnn_utils.py`파일에 선언되어 있으며, 앞서 불러온 바 있습니다.
* 행렬 곱셈의 경우 [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) 함수를 사용하세요.

In [ ]:
# GRADED FUNCTION: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = None
    # compute output of the current cell using the formula given above
    yt_pred = None
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [ ]:
np.random.seed(1)
xt_tmp = np.random.randn(3,10)
a_prev_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5,5)
parameters_tmp['Wax'] = np.random.randn(5,3)
parameters_tmp['Wya'] = np.random.randn(2,5)
parameters_tmp['ba'] = np.random.randn(5,1)
parameters_tmp['by'] = np.random.randn(2,1)

a_next_tmp, yt_pred_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, parameters_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = \n", a_next_tmp.shape)
print("yt_pred[1] =\n", yt_pred_tmp[1])
print("yt_pred.shape = \n", yt_pred_tmp.shape)

**모범 답안**: 
```Python
a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape = 
 (2, 10)

```

### 1.2 - RNN forward pass 

- RNN (Recurrent Neural Network)은 바로 이전 단계에서 구현한 RNN cell의 반복입니다.
  - 입력 데이터 시퀀스가 10개의 time step을 가지는경우 RNN cell을 10번 반복합니다.
- 각 cell은 각 time step에서 두 가지 입력을 받습니다.
  - $ a ^ {\langle t-1 \rangle} $ : 이전 cell의 hidden unit.
  - $ x ^ {\langle t \rangle} $ : 현재 time step의 입력 데이터.
- 마찬가지로 각 time step마다 두 개의 출력이 있습니다.
  - hidden unit ($ a ^ {\langle t \rangle} $)
  - 예측 결과 ($ y ^ {\langle t \rangle} $)
- 가중치 및 bias $ (W_ {aa}, b_ {a}, W_ {ax}, b_ {x}) $는 각 time step에서 재사용됩니다.
  - `parameters` 딕셔너리는 `rnn_cell_forward` 함수가 호출되는 사이에도 값이 유지됩니다.


<img src="arts/rnn_forward_sequence_figure3_v3a.png" style="width:800px;height:180px;">

<center>그림 3 : 기본적인 RNN 구조. 입력 시퀀스 $ x = (x ^ {\langle 1 \rangle}, x ^ {\langle 2 \rangle}, ..., x ^ {\langle T_x \rangle}) $는 $ T_x $ 각 time step에 걸쳐 전달됩니다. 인공신경망은 $ y = (y ^ {\langle 1 \rangle}, y ^ {\langle 2 \rangle}, ..., y ^ {\langle T_x \rangle}) $를 출력합니다.</center>


**연습 문제** : 그림 3에 설명 된 RNN의 forward propagation을 구현하세요.

**지시 사항**:
- RNN에 의해 ​​계산 된 모든 hidden unit를 저장할, shape $(n_{a}, m, T_ {x})$를 가지는 3D 배열 $a$를 만들고 이를 0으로 초기화합니다.
- 예측 결과를 저장할 $ (n_ {y}, m, T_ {x}) $ shape의 3D 배열 $ \hat {y} $를 만들고 이를 0으로 초기화합니다.
  - 이 경우 $ T_ {y} = T_ {x} $ (예측 결과 및 입력의 time step 수가 동일 함)에 유의하십시오.
- 최초의 hidden state $ a_ {0} $와 같은 값으로 2D hidden state `a_next`를 초기화합니다.
- 매 $t$ time step마다 :
  - 단일 time step $ t $에 대한 $ x $의 2D 슬라이스 인 $ x ^ {\langle t \rangle} $를 가져옵니다.
    - $ x ^ {\langle t\rangle} $의 shape는 $ (n_ {x}, m) $입니다.
    - $ x $의 shape는 $ (n_ {x}, m, T_ {x}) $입니다.
  - `rnn_cell_forward`를 실행하여 2D hidden state $ a ^ {\langle t \rangle} $ (변수 이름 = `a_next`), 예측 결과 $ \hat {y} ^ {\langle t \rangle} $ 및 캐시를 업데이트합니다. .
    - $ a ^ {\ langle t \ rangle} $ 모양이 $ (n_ {a}, m) $
  - $ t ^ {th} $ 번째 위치의 3D 텐서 $ a $에 2D hidden state를 저장합니다.
    - $ a $는 $ (n_ {a}, m, T_ {x}) $의 shape를 가집니다.
  - $ t $ 번째 위치에 3D 텐서 $ \hat {y} _ {pred} $에 2D $ \hat {y} ^ {\langle t \rangle} $ 예측 (변수 이름`yt_pred`)을 저장합니다.
    - $ \hat {y} ^ {\langle t \rangle} $의 shape는 $ (n_ {y}, m) $입니다.
    - $ \hat {y} $의 shape는 $ (n_ {y}, m, T_x) $입니다.
  - 캐시 목록에 캐시를 추가합니다.
- 3D 텐서 $ a $ 및 $ \hat {y} $와 캐시를 반환합니다.


**추가 힌트**
- [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
- 3 차원 numpy 배열이 있고 3 차원으로 인덱싱하는 경우 `var_name [:, :, i]` 과 같이 배열 슬라이싱을 사용할 수 있습니다. 


In [ ]:
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y_pred" with zeros (≈2 lines)
    a = None
    y_pred = None
    
    # Initialize a_next (≈1 line)
    a_next = None
    
    # loop over all time-steps of the input 'x' (1 line)
    for t in range(None):
        # Update next hidden state, compute the prediction, get the cache (≈2 lines)
        xt = None
        a_next, yt_pred, cache = None
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = None
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = None
        # Append "cache" to "caches" (≈1 line)
        None
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [ ]:
np.random.seed(1)
x_tmp = np.random.randn(3,10,4)
a0_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5,5)
parameters_tmp['Wax'] = np.random.randn(5,3)
parameters_tmp['Wya'] = np.random.randn(2,5)
parameters_tmp['ba'] = np.random.randn(5,1)
parameters_tmp['by'] = np.random.randn(2,1)

a_tmp, y_pred_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][1] = \n", a_tmp[4][1])
print("a.shape = \n", a_tmp.shape)
print("y_pred[1][3] =\n", y_pred_tmp[1][3])
print("y_pred.shape = \n", y_pred_tmp.shape)
print("caches[1][1][3] =\n", caches_tmp[1][1][3])
print("len(caches) = \n", len(caches_tmp))

**모범 답안**:

```Python
a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = 
 (5, 10, 4)
y_pred[1][3] =
 [ 0.79560373  0.86224861  0.11118257  0.81515947]
y_pred.shape = 
 (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 
 2
```

축하합니다! Recurrent Neural Network의 forward propagation을 성공적으로 구현했습니다.

#### Situations when this RNN will perform better:
- 위 구현의 경우 그라디언트가 사라지는 문제(Gradient vanishing)가 있습니다.
- RNN은 각 출력 $ \hat {y} ^ {\langle t \rangle} $을 "local" 컨텍스트를 사용하여 추정 할 수있을 때 가장 잘 작동합니다.
- "local" 컨텍스트는 예측의 time step $ t $에 가까운 정보를 나타냅니다.
- 보다 공식적으로, local 컨텍스트는 입력 $ x ^ {\langle t '\rangle} $ 및 예측 $ \hat {y} ^ {\langle t \rangle} $를 참조합니다. 여기서 $t'$는 $t$에 가깝습니다.

다음 부분에서는 더 복잡한 LSTM 모델을 구축 할 것입니다. LSTM은 정보를 더 잘 기억하고 여러 시간 단계 동안 저장해 둘 수 있습니다.

## 2 - Long Short-Term Memory (LSTM) network

다음 그림은 LSTM 셀의 작동을 보여줍니다.

<img src="arts/LSTM_figure4_v3a.png" style="width : 500; height : 400px;">
<center>그림 4 : LSTM-cell. 이는 모든 time step에서 "셀 상태" 또는 메모리 변수 $ c ^ {\langle t \rangle} $을 추적하고 업데이트하며, 이 값은 $a ^ {\langle t \rangle}$와 다를 수 있습니다.
$ softmax ^ {*} $에는 dense layer와 softmax가 포함되어 있습니다.</center>

위의 기본적인 RNN을 구현했던 방법과 비슷하게 단일 time step에 대해 LSTM 셀을 구현하는 것으로 시작합니다. 그런 다음 반복문내에서 그 함수를 반복적으로 호출하여 $ T_x $ time step에 대해 주어진 입력을 처리하도록 확장하겠습니다.

### Overview of gates and states

#### Forget gate $\mathbf{\Gamma}_{f}$
- 텍스트에서 단어를 읽고 문법 구조를 추적하기 위해 LSTM을 사용할 계획이라고 가정해 보겠습니다. 예를 들어 특정 명사가 단수 ("강아지")인지 복수 ("강아지")인지 인식하는 것입니다.
- 주어의 상태가 변경됨면 (단수형에서 복수형으로) 이전 상태를 폐기하고, 그 값을 잊어버립니다.
- "Forget Gate"는 0과 1 사이의 값을 포함하는 텐서입니다.
  - Forget gate에 있는 유닛의 값이 0에 가까운 경우, LSTM은 이전 셀 상태의 해당 유닛에 저장된 상태를 "잊습니다".
  - Forget gate에 있는 장치의 값이 1에 가까운 경우 LSTM은 대부분 저장된 상태 그대로의 값을 기억합니다.

##### Equation
$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$

##### Explanation of the equation:

- $ \mathbf {W_ {f}} $에는 forget gate의 동작을 제어하는 ​​가중치가 포함되어 있습니다.
- 이전 time step의 hidden state $ [a ^ {\langle t-1 \rangle} $ 와, 현재 time step의 입력 $ x ^ {\langle t \rangle}] $을 더하고 $ \mathbf{W_{f}}$를 곱합니다.
- 시그모이드 함수는 각 gate 텐서의 값 $ \mathbf {\Gamma} _f ^ {\langle t \rangle} $ 범위를 0에서 1까지 만드는 데 사용됩니다.
- Forget gate $ \mathbf {\Gamma} _f ^ {\langle t \rangle} $은 이전 셀 상태 $ c ^ {\langle t-1 \rangle} $과 shape가 동일합니다.
- 이것은 두 행렬을 요소별로 곱할 수 있음을 의미합니다.
- 텐서 $ \mathbf {\Gamma} _f ^ {\langle t \rangle} * \mathbf {c} ^ {\langle t-1 \rangle} $은 이전 셀 상태에 마스크를 적용하는 것과 같습니다.
- $ \mathbf {\Gamma} _f ^ {\langle t \rangle} $의 값이 0이거나 0에 가까우면 곱셈 결과 역시 0에 가깝습니다.
  - 이렇게 하면 $ \mathbf {c} ^ {\langle t-1 \rangle} $에 저장된 정보가 다음 time step으로 전달되지(기억되지) 않습니다.
- 마찬가지로 한 값이 1에 가까우면 곱셈 결과는 이전 셀 상태의 원래 값에 가깝습니다.
  - LSTM은 다음 시간 단계에서 사용할 $ \mathbf {c} ^ {\langle t-1 \rangle} $의 정보를 유지합니다(기억합니다).

##### Variable names in the code
코드의 변수 이름은 방정식과 유사합니다.
- `Wf` : Forget gate 가중치 $ \mathbf {W} _ {f} $
- `bf` : Forget gate bias $ \mathbf {b} _ {f} $
- `ft` : Forget gate $ \Gamma_f ^ {\langle t \rangle} $


#### Candidate value $\tilde{\mathbf{c}}^{\langle t \rangle}$

- 후보 값은 현재 셀 상태 $ \mathbf {c} ^ {\langle t \rangle} $에 **저장 될 수있는** 현재 time step의 정보를 포함하는 텐서입니다.
- 후보 값의 어느 부분이 전달되는지는 Update gate에 따라 다릅니다.
- 후보 값은 -1에서 1 사이의 값을 포함하는 텐서입니다.
- 물결표 "~(tilda)"는 후보 값 $ \tilde {\mathbf {c}} ^ {\langle t \rangle} $를 셀 상태 $ \mathbf {c} ^ {\langle t \rangle} $와 구별하기 위해 사용됩니다.

##### Equation
$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$

##### Explanation of the equation
- 'tanh' 함수는 -1 과 +1 사이의 값을 리턴합니다. 

##### Variable names in the code
- `cct`: 후보 값 $\mathbf{\tilde{c}}^{\langle t \rangle}$

#### Update gate $\mathbf{\Gamma}_{i}$
- Update gate를 사용하여 후보 값인 $\tilde{\mathbf{c}}^{\langle t \rangle}$의 값을  셀 상태(메모리 변수) $c^{\langle t \rangle}$에 얼마만큼 반영할지 결정합니다.
- Update gate는 후보 값 텐서 $ \tilde {\mathbf {c}} ^ {\langle t \rangle} $에서 셀 상태 $ \mathbf {c} ^ {\langle t \rangle}$로 전달되는 부분을 결정합니다.
- Update gate는 0과 1 사이의 값을 가지는 텐서입니다.
  - Update gate의 단위가 1에 가까우면, 후보 값 $\tilde{\mathbf{c}}^{\langle t \rangle}$d의 값이 hidden state $ \mathbf {c} ^ {\langle t \rangle}$로 더 많이 전달됩니다.
  - Update gate의 단위가 0에 가까우면, 후보 값이 hidden state로 전달되는 것을 방지합니다.
- 참고 문헌에 사용 된 표기법을 따르기 위해 "u"가 아닌 아래 첨자 "i"를 사용합니다.

##### Equation
$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 

##### Explanation of the equation
- Forget gate와 비슷하게, $ \mathbf{\Gamma} _i ^ {\langle t \rangle} $에서 시그모이드 함수는 0과 1 사이의 값을 출력합니다.
- Update gate 값은 후보 값과 요소별로 곱하여($\mathbf{\Gamma}_{i}^{\langle t \rangle} * \tilde{c}^{\langle t \rangle}$), 셀 상태 $\mathbf{c}^{\langle t \rangle}$를 결정하는데 사용됩니다.

##### Variable names in code (Please note that they're different than the equations)
- `Wi` : Update gate 가중치 $\mathbf{W}_i$ (not "Wu") 
- `bi` : Update gate bias $\mathbf{b}_i$ (not "bu")
- `it` : Forget gate $\mathbf{\Gamma}_i^{\langle t \rangle}$ (not "ut")

#### - Cell state $\mathbf{c}^{\langle t \rangle}$
- 셀 상태는 미래의 시간 단계로 전달되는 "메모리"입니다.
- 새 셀 상태 $ \mathbf {c} ^ {\langle t \rangle} $은 이전 셀 상태와 후보 값의 조합입니다.

##### Equation
$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$

##### Explanation of equation
- 이전 셀 상태 $ \mathbf {c} ^ {\langle t-1 \rangle} $은 Forget gate $ \mathbf {\Gamma} _ {f} ^ {\langle t \rangle}$에 의해 조정(가중)됩니다.
- 또한 후보 값 $\tilde{\mathbf{c}}^{\langle t \rangle}$는 Update gate $\mathbf{\Gamma}_{i}^{\langle t \rangle}$에 의하여 조정(가중)됩니다.

##### Variable names and shapes in the code
* `c`: 모든 time step에 대한 cell 상태, $\mathbf{c}$ shape $(n_{a}, m, T)$
* `c_next`: 다가오는 time step에 대한 cell 상태, $\mathbf{c}^{\langle t \rangle}$ shape $(n_{a}, m)$
* `c_prev`: 이전 time step에 대한 cell 상테, $\mathbf{c}^{\langle t-1 \rangle}$, shape $(n_{a}, m)$

#### Output gate $\mathbf{\Gamma}_{o}$
* Output gate는 time step의 예측 (출력)으로 전송되는 내용을 결정합니다.
* Output gate는 다른 게이트와 같습니다. 0에서 1 사이의 값을 포함합니다.

##### Equation

$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$ 

##### Explanation of the equation
* Output gate는 이전 hidden state $ \mathbf {a} ^ {\langle t-1 \rangle} $ 및 현재 입력 $ \mathbf {x} ^ {\langle t \rangle}$에 의해 결정됩니다.
* 시그모이드 함수는 Output gate의 범위를 0에서 1까지로 만듭니다.

##### Variable names in the code
- `Wo` : Output gate 가중치, $ \mathbf {W_o} $
- `bo` : Output gate bias, $ \mathbf {b_o} $
- `ot` : Output gate, $ \mathbf {\Gamma} _ {o} ^ {\langle t \rangle}$

#### Hidden state $\mathbf{a}^{\langle t \rangle}$
- Hidden state는 LSTM 셀의 다음 시간 단계로 전달됩니다.
- 다음 time step의 세가지 gate ($ \mathbf {\Gamma} _ {f}, \mathbf {\Gamma} _ {u}, \mathbf {\Gamma} _ {o} $)를 계산하는 데 사용됩니다.
- Hidden state는 예측 결과인 $ y ^ {\langle t \rangle} $를 계산하는 데에도 사용됩니다.

##### Equation
$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

##### Explanation of equation
- Hidden state $ \mathbf {a} ^ {\langle t \rangle} $는 cell state $\mathbf{c}^{\langle t \rangle}$와 output gate $\mathbf{\Gamma}_{o}$가 결합된 값에 의해 결정됩니다.
- Cell state는 "tanh"함수를 통해 전달되어 -1과 +1 사이의 값으로 재조정됩니다.
- Ouptut gate는 $\tanh(\mathbf{c}^{\langle t \rangle})$의 값을 보존하거나 이러한 값이 숨겨진 상태 $\mathbf{a}^{\langle t \rangle}$에 포함되지 않도록 하는 "마스크"와 같은 역할을 수행합니다.

##### Variable names  and shapes in the code
* `a`: 모든 time step을 포괄하는 hidden state.  $\mathbf{a}$의 shape는 $(n_{a}, m, T_{x})$입니다.
* 'a_prev`: 이전 time step의 hidden state. $\mathbf{a}^{\langle t-1 \rangle}$의 shape는 $(n_{a}, m)$입니다.
* `a_next`: 현재 time step의 hidden state. $\mathbf{a}^{\langle t \rangle}$의 shape는 $(n_{a}, m)$입니다.

#### Prediction $\mathbf{y}^{\langle t \rangle}_{pred}$
* 이번 과제에서의 예측 결과는 일종의 분류이므로 softmax 함수를 사용합니다.

공식은 다음과 같습니다.:
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

##### Variable names and shapes in the code
* `y_pred`: 모든 time step에 대한 예측 결과. $\mathbf{y}_{pred}$ has shape $(n_{y}, m, T_{x})$.  $(T_{y} = T_{x})$임에 주의하세요.
* `yt_pred`: 현재 time step에 대한 예측 결과 $t$. $\mathbf{y}^{\langle t \rangle}_{pred}$의 shape는 $(n_{y}, m)$입니다.

### 2.1 - LSTM cell

**연습 문제** : 그림 4에 설명 된 LSTM cell을 구현합니다.

**지시 사항**:
1. Hidden state $ a ^ {\langle t-1 \rangle} $와 입력 $ x ^ {\langle t \rangle} $를 단일 행렬로 연결합니다.
$$ concat = \begin {bmatrix} a ^ {\langle t-1 \rangle} \\ x ^ {\langle t \rangle} \end {bmatrix} $$
2. Gate, hidden state 및 cell state에 대한 모든 공식 1 ~ 6을 계산합니다.
3. $ y ^ {\langle t \rangle} $ 예측을 계산합니다.

#### Additional Hints

* [numpy.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html)를 사용할 수 있습니다. `axis` 매개 변수에 사용할 값을 확인하십시오.
* `sigmoid ()` 및 `softmax` 함수는 `rnn_utils.py`에서 가져옵니다.
* [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)
* 행렬 곱셈에는 [np.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)을 사용합니다.
* 변수 이름 `Wi`, `bi`는 **update** 게이트의 가중치와 편향을 나타냅니다. 이 함수에는 "Wu"또는 "bu"라는 변수가 없습니다.

In [ ]:
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo --  Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the cell state (memory)
    """

    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"] # forget gate weight
    bf = parameters["bf"]
    Wi = parameters["Wi"] # update gate weight (notice the variable name)
    bi = parameters["bi"] # (notice the variable name)
    Wc = parameters["Wc"] # candidate value weight
    bc = parameters["bc"]
    Wo = parameters["Wo"] # output gate weight
    bo = parameters["bo"]
    Wy = parameters["Wy"] # prediction weight
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ### START CODE HERE ###
    # Concatenate a_prev and xt (≈1 line)
    concat = None

    # Compute values for ft (forget gate), it (update gate),
    # cct (candidate value), c_next (cell state), 
    # ot (output gate), a_next (hidden state) (≈6 lines)
    ft = None        # forget gate
    it = None        # update gate
    cct = None       # candidate value
    c_next = None    # cell state
    ot = None        # output gate
    a_next = None    # hidden state
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = None
    ### END CODE HERE ###

    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [ ]:
np.random.seed(1)
xt_tmp = np.random.randn(3,10)
a_prev_tmp = np.random.randn(5,10)
c_prev_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5+3)
parameters_tmp['bf'] = np.random.randn(5,1)
parameters_tmp['Wi'] = np.random.randn(5, 5+3)
parameters_tmp['bi'] = np.random.randn(5,1)
parameters_tmp['Wo'] = np.random.randn(5, 5+3)
parameters_tmp['bo'] = np.random.randn(5,1)
parameters_tmp['Wc'] = np.random.randn(5, 5+3)
parameters_tmp['bc'] = np.random.randn(5,1)
parameters_tmp['Wy'] = np.random.randn(2,5)
parameters_tmp['by'] = np.random.randn(2,1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, parameters_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = ", a_next_tmp.shape)
print("c_next[2] = \n", c_next_tmp[2])
print("c_next.shape = ", c_next_tmp.shape)
print("yt[1] =", yt_tmp[1])
print("yt.shape = ", yt_tmp.shape)
print("cache[1][3] =\n", cache_tmp[1][3])
print("len(cache) = ", len(cache_tmp))

**모범 답안**:

```Python
a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
yt.shape =  (2, 10)
cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
```

### 2.2 - Forward pass for LSTM

이제 LSTM의 한 단계를 구현 했으므로 반복문을 사용하여 단계를 반복하여 $ T_x $의 입력 시퀀스를 처리 할 수 있습니다.

<img src="arts/LSTM_rnn.png" style="width:500;height:300px;">
<center>그림 5 : 여러 시간 단계에 걸친 LSTM.</center>

**연습 문제:** `lstm_forward()` 를 구현하여 $ T_x $의 time step에 걸쳐 LSTM을 실행합니다.

**지시 사항**
- 변수`x` 및 `parameters`의 shape를 참조해 $ n_x, n_a, n_y, m, T_x $ 차원을 가져옵니다.
- 3D 텐서 $ a $, $ c $ 및 $ y $를 초기화합니다.
  - $ a $ : hidden state, shape = $ (n_ {a}, m, T_ {x}) $
  - $ c $ : cell state, shape = $ (n_ {a}, m, T_ {x}) $
  - $ y $ : prediction, shape =  $ (n_ {y}, m, T_ {x}) $ (이 예에서 $ T_ {y} = T_ {x} $에 유의하십시오).
  - **참고** 한 변수를 다른 변수와 동일하게 설정하는 것은 "copy by reference" 입니다. 즉,`c = a`의 코드를 사용해서는 안됩니다. 이렇게 코드를 작성하게 되면 두 변수가 동일한 기본 변수를 참조하게 됩니다.
- 2D 텐서 $ a ^ {\langle t \rangle} $를 초기화 하세요.
  - $ a ^ {\langle t \rangle} $은 time step $ t $에 대한 hidden state를 저장합니다. 변수 이름은 `a_next`입니다.
  - $ a ^ {\langle 0 \rangle} $, 0번째 time step에서 hidden state 초기값입니다. 함수 호출시 전달됩니다. 변수 이름은 `a0`입니다.
  - $ a ^ {\langle t \rangle} $ 및 $ a ^ {\langle 0 \rangle} $는 단일 time step을 나타내므로 둘 다 $ (n_ {a}, m) $ shape를 갖습니다.
  - 함수에 전달되는 초기 hidden state ($ a ^ {\langle 0 \rangle} $)로 설정하여 $ a ^ {\langle t \rangle} $을 초기화합니다.
- $ c ^ {\langle t \rangle} $을 0으로 초기화합니다.
  - 변수 명은 `c_next`입니다.
  - $ c ^ {\langle t \rangle} $는 단일 time step을 나타내므로 shape는 $ (n_ {a}, m) $입니다.
  - **참고** :`c_next`를 메모리에 스스로의 영역이 있는 자체 변수로 선언하세요. 3D 텐서 $c$의 일부분으로 초기화해선 안됩니다. 다시 말해, `c_next = c [:, :, 0]` 과 같이 코드를 작성**하지 마세요**.
- 각 time step에 대해 다음을 수행하십시오.
  - 3D 텐서 $ x $에서 $ t $ time step에서 2D 슬라이스 $ x ^ {\langle t \rangle} $를 얻습니다.
  - 이전에 정의한 `lstm_cell_forward` 함수를 호출하여 hidden state, cell state, prediction, cache를 가져옵니다.
  - hidden state, cell state, prediction (2D 텐서)을 3D 텐서 안에 저장합니다.
  - 또한 캐시 목록에 캐시를 추가합니다.

In [ ]:
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (4).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    c -- The value of the cell state, numpy array of shape (n_a, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    ### START CODE HERE ###
    Wy = parameters['Wy'] # saving parameters['Wy'] in a local variable in case students use Wy instead of parameters['Wy']
    # Retrieve dimensions from shapes of x and parameters['Wy'] (≈2 lines)
    n_x, m, T_x = None
    n_y, n_a = None
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    a = None
    c = None
    y = None
    
    # Initialize a_next and c_next (≈2 lines)
    a_next = None
    c_next = None
    
    # loop over all time-steps
    for t in range(None):
        # Get the 2D slice 'xt' from the 3D input 'x' at time step 't'
        xt = None
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        a_next, c_next, yt, cache = None
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = None
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = None
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = None
        # Append the cache into caches (≈1 line)
        None
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return a, y, c, caches

In [ ]:
np.random.seed(1)
x_tmp = np.random.randn(3,10,7)
a0_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5+3)
parameters_tmp['bf'] = np.random.randn(5,1)
parameters_tmp['Wi'] = np.random.randn(5, 5+3)
parameters_tmp['bi']= np.random.randn(5,1)
parameters_tmp['Wo'] = np.random.randn(5, 5+3)
parameters_tmp['bo'] = np.random.randn(5,1)
parameters_tmp['Wc'] = np.random.randn(5, 5+3)
parameters_tmp['bc'] = np.random.randn(5,1)
parameters_tmp['Wy'] = np.random.randn(2,5)
parameters_tmp['by'] = np.random.randn(2,1)

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][3][6] = ", a_tmp[4][3][6])
print("a.shape = ", a_tmp.shape)
print("y[1][4][3] =", y_tmp[1][4][3])
print("y.shape = ", y_tmp.shape)
print("caches[1][1][1] =\n", caches_tmp[1][1][1])
print("c[1][2][1]", c_tmp[1][2][1])
print("len(caches) = ", len(caches_tmp))

**모범 답안**:

```Python
a[4][3][6] =  0.172117767533
a.shape =  (5, 10, 7)
y[1][4][3] = 0.95087346185
y.shape =  (2, 10, 7)
caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.855544916718
len(caches) =  2
```

축하합니다! 이제 기본 RNN 및 LSTM에 대한 forward propagation을 구현했습니다. 딥 러닝 프레임 워크를 사용하여 forward propagation를 구현하면 뛰어난 성능의 시스템을 구축할 수 있습니다.

아래 부분은 선택 사항이며 채점되지 않습니다.

## 3 - Backpropagation in recurrent neural networks (OPTIONAL / UNGRADED)

최신 딥 러닝 프레임 워크는 forward pass만 구현하면 알아서 backward pass를 처리하므로, 대부분의 딥 러닝 엔지니어는 backward pass의 세부 사항에 신경 쓸 필요가 없습니다. 하지만 만약 여러분이 미적분에 대한 세부 지식을 알고 있고, RNN backprop의 구체적인 구현 방법을 확인해보고 싶다면 노트북의 이 선택 과제를 수행하면 됩니다.

우리는 [이전 강좌](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/0VSHe/derivatives-with-a-computation-graph)에서 간단한 Fully connected 함수를 구현한 바 있습니다. 그 과정에서 비용 함수를 최소화시킬 수 있는 파라미터를 찾기 위해, backward propagation을 사용하여 대한 미분계수를 계산했습니다. 마찬가지로 RNN에서는 파라미터를 업데이트하기 위해 비용함수에 대한 미분계수를 계산할 수 있습니다. backward propagation 공식은 매우 복잡하기 때문에 동영상 강의에서 유도하지 않았습니다. 그러나 아래 파트에서 간략하게 설명하겠습니다.

이 노트북은 손실 'J'에서 'a'로의 backward path를 구현하지 않습니다. 여기에는 forward path의 일부인 dense layer와 softmax가 포함되었을 것입니다. 이것은 다른 곳에서 계산되고 결과가 'da'의 rnn_backward에 전달되는 것으로 간주됩니다. 또한 손실은 배치 크기(m)에 대해 사전에 조정되었으며 여기에서는 훈련 세트에 의한 분할이 필요하지 않다고 가정합니다.

이 파트는 선택 사항이며 채점되지 않습니다. 이 파트는 다른 부분보다 더 어렵고, 구현에 관한 세부 사항이 적습니다. 이 섹션은 전체 경로의 핵심 요소 만 구현합니다.

### 3.1 - Basic RNN backward pass

기본적인 RNN-셀에 대한 backward path를 계산하는 것으로 시작하고 다음 섹션에서는 해당 셀에 대한 연산을 반복합니다.

<img src="arts/rnn_backward_overview_3a_1.png" style="width:500;height:300px;">
<center>그림 6 : RNN- 셀의 backward path. Fully connected layer에서와 마찬가지로 비용 함수 $ J $의 미분은 미적분의 연쇄 법칙을 따라 RNN의 time step을 통해 역 전파됩니다. 셀 내부에서 chain-rule은 $(\frac {\partial J} {\partial W_ {ax}}, \frac {\partial J} {\partial W_ {aa}}, \frac {\partial J} {\partial b})$를 계산하는데도 사용됩니다. 매개 변수 $(W_ {ax}, W_ {aa}, b_a)$ 를 업데이트하기 위해 forward path의 캐시 된 결과를 활용할 수 있습니다.</center>


강의를 떠올려 보면 변수에 대한 비용의 편미분값은 dVariable입니다. 예를 들어 $ \frac {\partial J} {\partial W_ {ax}} $는 $ dW_ {ax} $입니다. 이 표기법은 아래의 섹션에서 사용됩니다.

<img src="images/rnn_cell_backward_3a_c.png" style="width:800;height:500px;"> 

<br>

<center>그림 7 : 이 rnn_cell_backward 구현에는 rnn_cell_forward에 포함 된 출력 dense layer 및 softmax가 포함되지 않습니다.</center>

$ da_ {next} $는 $ \frac {\partial {J}} {\partial a ^ {\langle t \rangle}} $이며 이전 단계 및 현재 단계 출력 로직의 손실을 포함합니다. 녹색으로 표시된 추가 사항은 rnn_backward 구현의 일부입니다.

##### Equations

rnn_cell_backward를 계산하려면 다음 공식들을 사용할 수 있습니다. 직접 쓰면서 공식을 유도해보는 것은 좋은 연습 방법이 될 수 있습니다. 여기서 $ * $ 기호는 행렬의 요소 별 곱셈을 나타내며 해당 기호가 없으면 행렬곱을 나타냅니다.

\begin{align}
\displaystyle a^{\langle t \rangle} &= \tanh(W_{ax} x^{\langle t \rangle} + W_{aa} a^{\langle t-1 \rangle} + b_{a})\tag{-} \\[8pt]
\displaystyle \frac{\partial \tanh(x)} {\partial x} &= 1 - \tanh^2(x) \tag{-} \\[8pt]
\displaystyle  {dW_{ax}} &= (da_{next} * ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) )) x^{\langle t \rangle T}\tag{1} \\[8pt]
\displaystyle dW_{aa} &= (da_{next} * ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) ))  a^{\langle t-1 \rangle T}\tag{2} \\[8pt]
\displaystyle db_a& = \sum_{batch}( da_{next} * ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) ))\tag{3} \\[8pt]
\displaystyle dx^{\langle t \rangle} &= { W_{ax}}^T (da_{next} * ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) ))\tag{4} \\[8pt]
\displaystyle da_{prev} &= { W_{aa}}^T(da_{next} *  ( 1-\tanh^2(W_{ax}x^{\langle t \rangle}+W_{aa} a^{\langle t-1 \rangle} + b_{a}) ))\tag{5}
\end{align}


#### Implementing rnn_cell_backward

위의 공식을 구현하여 결과를 직접 계산할 수 있습니다. 그러나 위의 내용은 'dz'를 계산하고 연쇄 법칙을 사용하여 추가적으로 단순화 할 수 있습니다.
$ \tanh (W_ {ax} x ^ {\langle t \rangle} + W_ {aa} a ^ {\langle t-1 \rangle} + b_ {a}) $가 forward에서 이미 계산된 바 있고, 캐시에 저장되었다는 것을 기억하면 위 공식은 더 간단해질 수 있습니다.

`dba`를 계산하기 위해 위의 'batch'는 모든 'm'예제 (축 = 1)의 합계입니다. `keepdims = True` 옵션을 사용해야합니다.

코스 1의 [Derivatives with a computaional graph](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/0VSHe/derivatives-with-a-computation-graph) 부터 [BackwardPropagation Intution](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/6dDj7/backpropagation-intuition-optional)을 검토하는 것이 좋습니다. 

행렬 벡터에 대한 미분은[여기](http://cs231n.stanford.edu/vecDerivs.pdf)에 설명되어 있지만 위의 공식에는 필요한 변환이 포함되어 있습니다.

rnn_cell_backward는 $y \langle t \rangle $의 손실 계산을 포함하지 않습니다. 이것은 da_next를 계산하는 곳에 통합됩니다. 이는 dense_layer와 softmax를 모두 포함하는 rnn_cell_forward와 backward 사이의 작은 차이점입니다.

참고 : 코드에서 :

$\displaystyle dx^{\langle t \rangle}$ is represented by dxt,   
$\displaystyle d W_{ax}$ is represented by dWax,   
$\displaystyle da_{prev}$ is represented by da_prev,    
$\displaystyle dW_{aa}$ is represented by dWaa,   
$\displaystyle db_{a}$ is represented by dba,   
dz는 위에서 파생되지 않았지만 반복 계산을 단순화하기 위해 선택적으로 여러분이 직접 구해볼 수 있습니다.

In [ ]:
def rnn_cell_backward(da_next, cache):
    """
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradients of input data, of shape (n_x, m)
                        da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    
    # Retrieve values from cache
    (a_next, a_prev, xt, parameters) = cache
    
    # Retrieve values from parameters
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]

    ### START CODE HERE ###
    # compute the gradient of the loss with respect to z (optional) (≈1 line)
    dz = None

    # compute the gradient of the loss with respect to Wax (≈2 lines)
    dxt = None
    dWax = None

    # compute the gradient with respect to Waa (≈2 lines)
    da_prev = None
    dWaa = None

    # compute the gradient with respect to b (≈1 line)
    dba = None

    ### END CODE HERE ###
    
    # Store the gradients in a python dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
    
    return gradients

In [ ]:
np.random.seed(1)
xt_tmp = np.random.randn(3,10)
a_prev_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Wax'] = np.random.randn(5,3)
parameters_tmp['Waa'] = np.random.randn(5,5)
parameters_tmp['Wya'] = np.random.randn(2,5)
parameters_tmp['ba'] = np.random.randn(5,1)
parameters_tmp['by'] = np.random.randn(2,1)

a_next_tmp, yt_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, parameters_tmp)

da_next_tmp = np.random.randn(5,10)
gradients_tmp = rnn_cell_backward(da_next_tmp, cache_tmp)
print("gradients[\"dxt\"][1][2] =", gradients_tmp["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients_tmp["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients_tmp["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients_tmp["da_prev"].shape)
print("gradients[\"dWax\"][3][1] =", gradients_tmp["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients_tmp["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients_tmp["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients_tmp["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients_tmp["dba"][4])
print("gradients[\"dba\"].shape =", gradients_tmp["dba"].shape)

**모범 답안**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           -1.3872130506
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           -0.152399493774
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           0.410772824935
        </td>
    </tr>
            <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           1.15034506685
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [ 0.20023491]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

#### Backward pass through the RNN

모든 time step $ t $에서 $ a ^ {\langle t \rangle} $에 대한 비용 함수의 기울기를 계산하는 것은 해당 기울기가 이전 RNN 셀로 역전파 되는데 도움이 되기 때문에 유용합니다. 이렇게하려면 마지막 셀에서 시작하여 모든 time step을 반복해야하며 각 step에서 전체 $ db_a $, $ dW_ {aa} $, $ dW_ {ax} $를 증가시키고 $ dx $를 저장합니다.

**지시 사항**:

`rnn_backward` 함수를 구현합니다. 리턴 값을 먼저 0으로 초기화 한 다음 각 time step에서`rnn_cell_backward`를 호출하면서 모든 time step을 반복하고 그에 따라 다른 변수를 업데이트합니다.

* 이 노트북은 Loss 'J'에서 'a'로의 backward pass를 구현하지 않습니다.
  * 여기에는 forward pass의 일부인 dense layer와 softmax가 포함되었을 것입니다.
  * 이것은 다른 곳에서 계산되고 결과가 'da'의 rnn_backward에 전달되는 것으로 가정합니다.
  * rnn_cell_backward를 호출 할 때 이전 단계의 손실과 결합해야합니다 (위의 그림 7 참조).
* 또한 배치 크기 (m)에 대해 손실이 조정 된 것으로 가정합니다.
  * 따라서 여기에서는 훈련 세트 수로 나눌 필요가 없습니다.

In [ ]:
def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """
        
    ### START CODE HERE ###
    
    # Retrieve values from the first cache (t=1) of caches (≈2 lines)
    (caches, x) = None
    (a1, a0, x1, parameters) = None
    
    # Retrieve dimensions from da's and x1's shapes (≈2 lines)
    n_a, m, T_x = None
    n_x, m = None
    
    # initialize the gradients with the right sizes (≈6 lines)
    dx = None
    dWax = None
    dWaa = None
    dba = None
    da0 = None
    da_prevt = None
    
    # Loop through all the time steps
    for t in reversed(range(None)):
        # Compute gradients at time step t. 
        # Remember to sum gradients from the output path (da) and the previous timesteps (da_prevt) (≈1 line)
        gradients = None
        # Retrieve derivatives from gradients (≈ 1 line)
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        # Increment global derivatives w.r.t parameters by adding their derivative at time-step t (≈4 lines)
        dx[:, :, t] = None
        dWax += None
        dWaa += None
        dba += None
        
    # Set da0 to the gradient of a which has been backpropagated through all time-steps (≈1 line) 
    da0 = None
    ### END CODE HERE ###

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients

In [ ]:
np.random.seed(1)
x_tmp = np.random.randn(3,10,4)
a0_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Wax'] = np.random.randn(5,3)
parameters_tmp['Waa'] = np.random.randn(5,5)
parameters_tmp['Wya'] = np.random.randn(2,5)
parameters_tmp['ba'] = np.random.randn(5,1)
parameters_tmp['by'] = np.random.randn(2,1)

a_tmp, y_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, parameters_tmp)
da_tmp = np.random.randn(5, 10, 4)
gradients_tmp = rnn_backward(da_tmp, caches_tmp)

print("gradients[\"dx\"][1][2] =", gradients_tmp["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients_tmp["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients_tmp["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients_tmp["da0"].shape)
print("gradients[\"dWax\"][3][1] =", gradients_tmp["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients_tmp["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients_tmp["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients_tmp["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients_tmp["dba"][4])
print("gradients[\"dba\"].shape =", gradients_tmp["dba"].shape)

**모범 답안**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [-2.07101689 -0.59255627  0.02466855  0.01483317]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           -0.314942375127
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dWax"][3][1]** =
        </td>
        <td>
           11.2641044965
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWax"].shape** =
        </td>
        <td>
           (5, 3)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"][1][2]** = 
        </td>
        <td>
           2.30333312658
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWaa"].shape** =
        </td>
        <td>
           (5, 5)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"][4]** = 
        </td>
        <td>
           [-0.74747722]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dba"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

## 3.2 - LSTM backward pass

### 3.2.1 One Step backward

LSTM backward pass는 forward pass보다 약간 더 복잡합니다.

<img src="arts/LSTM_cell_backward_rev3a_c2.png" style="width:500;height:400px;">
<center>그림 8 : lstm_cell_backward. lstm_cell_forward의 일부이지만 출력 함수는 lstm_cell_backward에 포함되지 않습니다.</center>

LSTM backward pass에 대한 공식은 다음과 같습니다. (미적분 연습에 익숙한 경우 처음부터 직접 유도 해보십시오.)

#### 3.2.2 gate derivatives
dense layer와 활성화 함수 사이의 게이트 미분 ($ \gamma $ ..)의 위치를 확인합니다 (위의 그래픽 참조). 이것은 다음 단계에서 파라미터의 미분을 계산하는 데 편리합니다.

\begin{align}
d\gamma_o^{\langle t \rangle} &= da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*\left(1-\Gamma_o^{\langle t \rangle}\right)\tag{7} \\[8pt]
dp\widetilde{c}^{\langle t \rangle} &= \left(dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \Gamma_u^{\langle t \rangle} * da_{next} \right) * \left(1-\left(\widetilde c^{\langle t \rangle}\right)^2\right) \tag{8} \\[8pt]
d\gamma_u^{\langle t \rangle} &= \left(dc_{next}*\widetilde{c}^{\langle t \rangle} + \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \widetilde{c}^{\langle t \rangle} * da_{next}\right)*\Gamma_u^{\langle t \rangle}*\left(1-\Gamma_u^{\langle t \rangle}\right)\tag{9} \\[8pt]
d\gamma_f^{\langle t \rangle} &= \left(dc_{next}* c_{prev} + \Gamma_o^{\langle t \rangle} * (1-\tanh^2(c_{next})) * c_{prev} * da_{next}\right)*\Gamma_f^{\langle t \rangle}*\left(1-\Gamma_f^{\langle t \rangle}\right)\tag{10}
\end{align}

### 3.2.3 parameter derivatives 

$ dW_f = d\gamma_f^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{11} $
$ dW_u = d\gamma_u^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{12} $
$ dW_c = dp\widetilde c^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{13} $
$ dW_o = d\gamma_o^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{14}$

$ db_f, db_u, db_c, db_o $를 계산하려면 $d\gamma_f^{\langle t \rangle}, d\gamma_u^{\langle t \rangle}, dp\widetilde c^{\langle t \rangle}, d\gamma_o^{\langle t \rangle}$ 의 모든 'm'예제 (축 = 1)를 합산하면 됩니다. `keepdims = True` 옵션이 있어야합니다.

$\displaystyle db_f = \sum_{batch}d\gamma_f^{\langle t \rangle}\tag{15}$
$\displaystyle db_u = \sum_{batch}d\gamma_u^{\langle t \rangle}\tag{16}$
$\displaystyle db_c = \sum_{batch}d\gamma_c^{\langle t \rangle}\tag{17}$
$\displaystyle db_o = \sum_{batch}d\gamma_o^{\langle t \rangle}\tag{18}$

마지막으로, 이전 hidden state, 이전 메모리 상태 및 입력에 대한 미분을 계산합니다.

$ da_{prev} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T   d\gamma_u^{\langle t \rangle}+ W_c^T dp\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle} \tag{19}$

여기서 연결부를 설명하기 위해 19번 공식의 가중치는 첫 번째 n_a입니다 (예 : $ W_f = W_f [:, : n_a] $ 등 ...).

$ dc_{prev} = dc_{next}*\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh^2(c_{next}))*\Gamma_f^{\langle t \rangle}*da_{next} \tag{20}$

$ dx^{\langle t \rangle} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T  d\gamma_u^{\langle t \rangle}+ W_c^T dp\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle}\tag{21} $

공식 21의 가중치는 n_a에서 끝까지 입니다 (예 : $ W_f = W_f [:, n_a :] $ 등 ...)

<br>

**연습 문제 :** 위의 공식 $ 7-21 $을 사용하여 `lstm_cell_backward` 를 구현합니다.

    
참고 : 코드에서 :

$d\gamma_o^{\langle t \rangle}$ is represented by  `dot`,    
$dp\widetilde{c}^{\langle t \rangle}$ is represented by  `dcct`,  
$d\gamma_u^{\langle t \rangle}$ is represented by  `dit`,  
$d\gamma_f^{\langle t \rangle}$ is represented by  `dft`


In [ ]:
def lstm_cell_backward(da_next, dc_next, cache):
    """
    Implement the backward pass for the LSTM-cell (single time-step).

    Arguments:
    da_next -- Gradients of next hidden state, of shape (n_a, m)
    dc_next -- Gradients of next cell state, of shape (n_a, m)
    cache -- cache storing information from the forward pass

    Returns:
    gradients -- python dictionary containing:
                        dxt -- Gradient of input data at time-step t, of shape (n_x, m)
                        da_prev -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                        dc_prev -- Gradient w.r.t. the previous memory state, of shape (n_a, m, T_x)
                        dWf -- Gradient w.r.t. the weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        dWi -- Gradient w.r.t. the weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        dWc -- Gradient w.r.t. the weight matrix of the memory gate, numpy array of shape (n_a, n_a + n_x)
                        dWo -- Gradient w.r.t. the weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                        dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                        dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                        dbo -- Gradient w.r.t. biases of the output gate, of shape (n_a, 1)
    """

    # Retrieve information from "cache"
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
    
    ### START CODE HERE ###
    # Retrieve dimensions from xt's and a_next's shape (≈2 lines)
    n_x, m = None
    n_a, m = None
    
    # Compute gates related derivatives, you can find their values can be found by looking carefully at equations (7) to (10) (≈4 lines)
    dot = None
    dcct = None
    dit = None
    dft = None
    
    # Compute parameters related derivatives. Use equations (11)-(18) (≈8 lines)
    dWf = None
    dWi = None
    dWc = None
    dWo = None
    dbf = None
    dbi = None
    dbc = None
    dbo = None

    # Compute derivatives w.r.t previous hidden state, previous memory state and input. Use equations (19)-(21). (≈3 lines)
    da_prev = None
    dc_prev = None
    dxt = None
    ### END CODE HERE ###
    
    # Save gradients in dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

    return gradients

In [ ]:
np.random.seed(1)
xt_tmp = np.random.randn(3,10)
a_prev_tmp = np.random.randn(5,10)
c_prev_tmp = np.random.randn(5,10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5+3)
parameters_tmp['bf'] = np.random.randn(5,1)
parameters_tmp['Wi'] = np.random.randn(5, 5+3)
parameters_tmp['bi'] = np.random.randn(5,1)
parameters_tmp['Wo'] = np.random.randn(5, 5+3)
parameters_tmp['bo'] = np.random.randn(5,1)
parameters_tmp['Wc'] = np.random.randn(5, 5+3)
parameters_tmp['bc'] = np.random.randn(5,1)
parameters_tmp['Wy'] = np.random.randn(2,5)
parameters_tmp['by'] = np.random.randn(2,1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, parameters_tmp)

da_next_tmp = np.random.randn(5,10)
dc_next_tmp = np.random.randn(5,10)
gradients_tmp = lstm_cell_backward(da_next_tmp, dc_next_tmp, cache_tmp)
print("gradients[\"dxt\"][1][2] =", gradients_tmp["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients_tmp["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients_tmp["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients_tmp["da_prev"].shape)
print("gradients[\"dc_prev\"][2][3] =", gradients_tmp["dc_prev"][2][3])
print("gradients[\"dc_prev\"].shape =", gradients_tmp["dc_prev"].shape)
print("gradients[\"dWf\"][3][1] =", gradients_tmp["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients_tmp["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients_tmp["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients_tmp["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients_tmp["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients_tmp["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients_tmp["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients_tmp["dWo"].shape)
print("gradients[\"dbf\"][4] =", gradients_tmp["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients_tmp["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients_tmp["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients_tmp["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients_tmp["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients_tmp["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients_tmp["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients_tmp["dbo"].shape)

**모범 답안**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           3.23055911511
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           -0.0639621419711
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dc_prev"][2][3]** =
        </td>
        <td>
           0.797522038797
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dc_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.147954838164
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           1.05749805523
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           2.30456216369
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.331311595289
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [ 0.18864637]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.40142491]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [ 0.25587763]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ 0.13893342]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### 3.3 Backward pass through the LSTM RNN

이 부분은 위에서 구현 한 `rnn_backward` 함수와 매우 유사합니다. 먼저 리턴 값과 동일한 차원의 변수를 만듭니다. 그런 다음 끝부터 시작하여 모든 time step을 반복하고 각 반복에서 LSTM에 대해 구현 한 1 단계 함수를 호출합니다. 그런 다음 파라미터를 개별적으로 합산하여 업데이트합니다. 마지막으로 업데이트된 gradient가 있는 딕셔너리를 반환합니다.

**지시 사항** :`lstm_backward` 함수를 구현합니다. $ T_x $에서 시작하여 뒤로 돌아가는 for 루프를 만듭니다. 각 단계에 대해 `lstm_cell_backward`를 호출하고 새 그라디언트를 추가하여 이전 그라디언트를 업데이트하십시오. `dxt`는 업데이트되지 않고 저장됩니다.

In [ ]:
def lstm_backward(da, caches):
    
    """
    Implement the backward pass for the RNN with LSTM-cell (over a whole sequence).

    Arguments:
    da -- Gradients w.r.t the hidden states, numpy-array of shape (n_a, m, T_x)
    caches -- cache storing information from the forward pass (lstm_forward)

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient of inputs, of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                        dWf -- Gradient w.r.t. the weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        dWi -- Gradient w.r.t. the weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        dWc -- Gradient w.r.t. the weight matrix of the memory gate, numpy array of shape (n_a, n_a + n_x)
                        dWo -- Gradient w.r.t. the weight matrix of the save gate, numpy array of shape (n_a, n_a + n_x)
                        dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                        dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                        dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                        dbo -- Gradient w.r.t. biases of the save gate, of shape (n_a, 1)
    """

    # Retrieve values from the first cache (t=1) of caches.
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    
    ### START CODE HERE ###
    # Retrieve dimensions from da's and x1's shapes (≈2 lines)
    n_a, m, T_x = None
    n_x, m = None
    
    # initialize the gradients with the right sizes (≈12 lines)
    dx = None
    da0 = None
    da_prevt = None
    dc_prevt = None
    dWf = None
    dWi = None
    dWc = None
    dWo = None
    dbf = None
    dbi = None
    dbc = None
    dbo = None
    
    # loop back over the whole sequence
    for t in reversed(range(None)):
        # Compute all gradients using lstm_cell_backward
        gradients = None
        # Store or add the gradient to the parameters' previous step's gradient
        da_prevt = None
        dc_prevt = None
        dx[:,:,t] = None
        dWf += None
        dWi += None
        dWc += None
        dWo += None
        dbf += None
        dbi += None
        dbc += None
        dbo += None
    # Set the first activation's gradient to the backpropagated gradient da_prev.
    da0 = None
    
    ### END CODE HERE ###

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}
    
    return gradients

In [ ]:
np.random.seed(1)
x_tmp = np.random.randn(3,10,7)
a0_tmp = np.random.randn(5,10)

parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5+3)
parameters_tmp['bf'] = np.random.randn(5,1)
parameters_tmp['Wi'] = np.random.randn(5, 5+3)
parameters_tmp['bi'] = np.random.randn(5,1)
parameters_tmp['Wo'] = np.random.randn(5, 5+3)
parameters_tmp['bo'] = np.random.randn(5,1)
parameters_tmp['Wc'] = np.random.randn(5, 5+3)
parameters_tmp['bc'] = np.random.randn(5,1)
parameters_tmp['Wy'] = np.zeros((2,5))       # unused, but needed for lstm_forward
parameters_tmp['by'] = np.zeros((2,1))       # unused, but needed for lstm_forward

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, parameters_tmp)

da_tmp = np.random.randn(5, 10, 4)
gradients_tmp = lstm_backward(da_tmp, caches_tmp)

print("gradients[\"dx\"][1][2] =", gradients_tmp["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients_tmp["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients_tmp["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients_tmp["da0"].shape)
print("gradients[\"dWf\"][3][1] =", gradients_tmp["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients_tmp["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients_tmp["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients_tmp["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients_tmp["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients_tmp["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients_tmp["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients_tmp["dWo"].shape)
print("gradients[\"dbf\"][4] =", gradients_tmp["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients_tmp["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients_tmp["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients_tmp["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients_tmp["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients_tmp["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients_tmp["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients_tmp["dbo"].shape)

**모범 답안**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [0.00218254  0.28205375 -0.48292508 -0.43281115]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           0.312770310257
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.0809802310938
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           0.40512433093
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           -0.0793746735512
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.038948775763
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [-0.15745657]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.50848333]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [-0.42510818]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ -0.17958196]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Congratulations

이 과제를 완료 한 것을 축하합니다. 이제 순환 신경망이 어떻게 작동하는지 이해했습니다!

RNN을 사용하여 문자 수준 언어 모델을 빌드하는 다음 과제로 넘어가겠습니다.